### 🚕 Ride Duration Prediction – Batch Mode

This notebook applies a pre-trained model to NYC Yellow Taxi data (March 2023).
All results are retained and warnings are suppressed.


In [1]:
!pip freeze | findstr scikit-learn

scikit-learn==1.7.0


In [2]:
!python -V

Python 3.10.9


In [9]:
round(y_pred.std(), 3)

np.float64(6.247)

In [22]:
import os
size_bytes = os.path.getsize("output/result.parquet")
size_mb = size_bytes / (1024 * 1024)
print(f"File size: {size_bytes:,} bytes ({size_mb:.2f} MB)")

File size: 68,640,820 bytes (65.46 MB)


In [49]:
from json import load
with open("Pipfile.lock", "rb") as file_in:
    pipfile = load(file_in)
    print(pipfile["default"]["scikit-learn"]["hashes"][0])

sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c


In [56]:
!docker build -t ride-duration-prediction:v1 . > /dev/null 2>&1
!docker run --rm ride-duration-prediction:v1

The mean predicted duration is 0.189 minutes
